In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import msgpack
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import msgpack
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.mathTools.CurveAnalysis import CurveAnalysis
from src.predictionModule.CurveML import CurveML


In [11]:
stock = yf.Ticker("IRM")
fullSharePrice = stock.history(period="max")
#fullSharePrice[["Open", "High", "Low", "Close", "Adj Close"]]
fullSharePrice

c:\Users\KILightTouch\Desktop\RandomOdyssey\.venv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [3]:
from src.common.YamlTickerInOut import YamlTickerInOut

snp500tickers = YamlTickerInOut("../src/tickerSelection").loadFromFile("snp500.yaml")["snp500tickers"]
alltickers = YamlTickerInOut("../src/stockGroups").loadFromFile("group_all.yaml")

In [7]:
pd.Timestamp(None)

NaT

In [10]:
assetG = AssetFileInOut("../src/database").loadFromFile('NVO')
priceData = assetG.shareprice.iloc[-21:-1]["Close"]
priceData = priceData.resample('B').mean().dropna()

print(priceData.index[-1])

assets=AssetFileInOut("../src/stockGroups/bin").loadDictFromFile("group_swiss_over20years")

startDate=pd.Timestamp(year=2010,month=1,day=4, tz='UTC')
endDate=pd.Timestamp(year=2020,month=1,day=4, tz='UTC')

curveML = CurveML(assets, startDate, endDate)
curveML.loadModel("../src/predictionModule/bin", "curveML_swiss_10to20")

a=curveML.predictNextPrices(priceData, 'TEST',1)

print(priceData.values[-1])
print(type(a))


2024-10-02 00:00:00-04:00
Model loaded from ../src/predictionModule/bin\curveML_swiss_10to20.mdl.
116.4800033569336
<class 'numpy.ndarray'>
